In [1]:
import os
import re
from dataclasses import dataclass
from time import strftime
from typing import Optional, Dict, Union, Any

import numpy as np
import pandas as pd
from tqdm import tqdm
from sklearn.metrics import roc_auc_score

import datasets
from datasets import Dataset, load_dataset, concatenate_datasets
import evaluate

import torch
import transformers
from transformers import (
    AutoConfig,
    AutoModelForCausalLM,
    AutoModelForSequenceClassification,
    AutoTokenizer,
    DataCollatorForLanguageModeling,
    DataCollatorForSeq2Seq,
    DataCollatorWithPadding,
    GPTNeoXConfig,
    LlamaConfig,
    Seq2SeqTrainer,
    Seq2SeqTrainingArguments,
    T5Config,
    Trainer,
    TrainingArguments,
    logging as hf_logging,
)
from peft import LoraConfig, TaskType, get_peft_model

chrf_metric = evaluate.load("chrf")


In [ ]:
os.environ['CLEARML_API_ACCESS_KEY'] = '' # Enter your own
os.environ['CLEARML_API_SECRET_KEY'] = ''
os.environ['CLEARML_API_HOST'] = 'https://api.clear.ml'
os.environ['CLEARML_LOG_MODEL'] = 'FALSE'
os.environ['CLEARML_TASK'] = 'ru-bashkir-post'

In [ ]:
from transformers import AutoTokenizer
import datasets
from datasets import concatenate_datasets

MODEL = 'tencent/HY-MT1.5-7B'

tokenizer = AutoTokenizer.from_pretrained(MODEL)

def build_text_ru_chv(example):
    """
    Format English->Chuvash training examples using HY-MT1.5 chat template.
    """
    source_text = example['ru']
    target_text = example['ba']
    
    messages = [
        {
            "role": "user",
            "content": f"Translate the following segment into Bashkir, without additional explanation.\n\n{source_text}"
        },
        {
            "role": "assistant",
            "content": target_text
        }
    ]
    
    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=False
    )
    
    example["text"] = text
    return example

def build_text_ru_bk(example):
    """
    Format Russian->Chuvash training examples using HY-MT1.5 chat template.
    """
    source_text = example['ru']
    target_text = example['chv']
    
    messages = [
        {
            "role": "user",
            "content": f"Translate the following segment into Chuvash, without additional explanation.\n\n{source_text}"
        },
        {
            "role": "assistant",
            "content": target_text
        }
    ]
    
    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=False
    )
    
    example["text"] = text
    return example

def tokenize_example(example):
    """
    Tokenize the formatted text.
    """
    encoded = tokenizer(
        example["text"],
        truncation=True,
        max_length=512,
        padding=False
    )
    example["input_ids"] = encoded["input_ids"]
    example["attention_mask"] = encoded["attention_mask"]
    
    example["labels"] = encoded["input_ids"].copy()
    
    return example

ru_bk_full = datasets.load_dataset('AigizK/bashkir-russian-parallel-corpora')['train']


ru_bk_split = ru_bk_full.train_test_split(test_size=0.05, seed=42)
ru_bk_train = ru_bk_split['train']
ru_bk_test = ru_bk_split['test']

ru_bk_train = ru_bk_train.map(build_text_ru_bk, num_proc=8)
ru_bk_train = ru_bk_train.map(tokenize_example, batched=False, num_proc=16)

ru_bk_test = ru_bk_test.map(build_text_ru_bk, num_proc=8)
ru_bk_test = ru_bk_test.map(tokenize_example, batched=False, num_proc=16)

print(f"EN-CHV train: {len(ru_bk_train)} examples")
print(f"EN-CHV test: {len(ru_bk_test)} examples")

ru_bk_full = datasets.load_dataset('alexantonov/chuvash_russian_parallel')['train'].select(range(300_000))
ru_bk_split = ru_bk_full.train_test_split(test_size=0.05, seed=42)
ru_bk_train = ru_bk_split['train']
ru_bk_val = ru_bk_split['test']

ru_bk_train = ru_bk_train.map(build_text_ru_chv, num_proc=4)
ru_bk_train = ru_bk_train.map(tokenize_example, batched=False, num_proc=128)

ru_bk_val = ru_bk_val.map(build_text_ru_chv, num_proc=4)
ru_bk_val = ru_bk_val.map(tokenize_example, batched=False, num_proc=128)

train_combined = concatenate_datasets([ru_bk_train, ru_bk_train])
train_combined = train_combined.shuffle(seed=42)

ds = datasets.DatasetDict({
    'train': ru_bk_train,
    'test': ru_bk_test
})


ds['test'] = ds['test'].select(range(100))

print(f"\nTrain: {len(ds['train'])} examples (EN+RU -> BK, shuffled)")
print(f"Test: {len(ds['test'])} examples (EN -> BK)")

README.md:   0%|          | 0.00/344 [00:00<?, ?B/s]

data/train-00000-of-00001.parquet:   0%|          | 0.00/302M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/1461485 [00:00<?, ? examples/s]

Filter:   0%|          | 0/1461485 [00:00<?, ? examples/s]

Map (num_proc=8):   0%|          | 0/1381942 [00:00<?, ? examples/s]

Map (num_proc=16):   0%|          | 0/1381942 [00:00<?, ? examples/s]

Map (num_proc=8):   0%|          | 0/72734 [00:00<?, ? examples/s]

Map (num_proc=16):   0%|          | 0/72734 [00:00<?, ? examples/s]

EN-CHV train: 1381942 examples
EN-CHV test: 72734 examples

Train: 1381942 examples (EN+RU -> CHV, shuffled)
Test: 100 examples (EN -> CHV)


In [4]:
ds['train'] = ds['train'].remove_columns(['chv', 'en', 'text', 'ru'])
ds['test'] = ds['test'].remove_columns(['chv', 'en', 'text'])


In [ ]:
MODEL = 'tencent/HY-MT1.5-7B'

tokenizer = AutoTokenizer.from_pretrained(MODEL, padding_side='left')

if tokenizer.pad_tokru_id is None:
    tokenizer.pad_tokru_id = tokenizer.eos_tokru_id

model = AutoModelForCausalLM.from_pretrained(
    MODEL, 
    dtype=torch.bfloat16, 
).cuda()

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [ ]:
from peft import LoraConfig, TaskType, get_peft_model

target_modules = [
    "q_proj",
    "k_proj",
    "v_proj",
    "o_proj",
]

lora_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,
    inference_mode=False,
    r=16, 
    lora_alpha=32,
    lora_dropout=0.05,
    target_modules=target_modules,
)
model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

trainable params: 13,631,488 || all params: 7,518,199,808 || trainable%: 0.1813


In [ ]:
IS_7B = True
if IS_7B:
    ASSISTANT_TOKEN = 127962
else:
    ASSISTANT_TOKEN = 120007

def compute_metrics(eval_preds):
    preds, labels = eval_preds

    if isinstance(preds, tuple):
        preds = preds[0]

    # 2) If preds are logits: (bs, seq, vocab) -> take argmax
    if isinstance(preds, np.ndarray) and preds.ndim == 3:
        preds = np.argmax(preds, axis=-1)

    # 3) Replace -100 (or any negative ids) in preds and labels
    preds = np.where(preds < 0, tokenizer.pad_tokru_id, preds)
    labels = np.where(labels < 0, tokenizer.pad_tokru_id, labels)

    # 4) just to be sure
    preds = preds.astype(np.int32)
    labels = labels.astype(np.int32)

    # 5) Slice to keep only tokens after assistant token (not including it)
    truncated_preds = []
    truncated_labels = []
    
    for pred_seq, label_seq in zip(preds, labels):
        pred_asst_pos = np.where(pred_seq == ASSISTANT_TOKEN)[0]
        if len(pred_asst_pos) > 0:
            start_pos = pred_asst_pos[0] + 1
            truncated_preds.append(pred_seq[start_pos:])
        else:
            truncated_preds.append(pred_seq)
        
        label_asst_pos = np.where(label_seq == ASSISTANT_TOKEN)[0]
        if len(label_asst_pos) > 0:
            start_pos = label_asst_pos[0] + 1
            truncated_labels.append(label_seq[start_pos:])
        else:
            truncated_labels.append(label_seq)

    decoded_preds = tokenizer.batch_decode(truncated_preds, skip_special_tokens=True)
    decoded_labels = tokenizer.batch_decode(truncated_labels, skip_special_tokens=True)

    decoded_preds = [p.strip() for p in decoded_preds]
    decoded_labels = [l.strip() for l in decoded_labels]
    
    print(f'Generated (first 3): {decoded_preds[:10]}')
    print(f'Labels (first 3): {decoded_labels[:10]}')
    
    result = chrf_metric.compute(
        predictions=decoded_preds,
        references=decoded_labels,
        word_order=2
    )
    return {"chrf": result["score"]}


class HYMTSeq2SeqTrainer(Seq2SeqTrainer):

    def _mask_prompt_labels(self, inputs):
        if "labels" not in inputs or "input_ids" not in inputs:
            return inputs

        input_ids = inputs["input_ids"]
        labels = inputs["labels"]
        if not torch.is_tensor(labels):
            return inputs

        labels = labels.clone()

        for i in range(input_ids.size(0)):
            pos = (input_ids[i] == ASSISTANT_TOKEN).nonzero(as_tuple=True)[0]
            if pos.numel() == 0:
                labels[i, :] = -100
            else:
                cut = int(pos[0].item()) + 1
                labels[i, :cut] = -100

        inputs["labels"] = labels
        return inputs

    def compute_loss(self, model, inputs, return_outputs=False, **kwargs):
        inputs = self._mask_prompt_labels(inputs)
        return super().compute_loss(model, inputs, return_outputs=return_outputs, **kwargs)

    def prediction_step(self, model, inputs, prediction_loss_only, ignore_keys=None, **gru_kwargs):
        has_labels = "labels" in inputs
        inputs = self._prepare_inputs(inputs)

        if has_labels:
            inputs = self._mask_prompt_labels(inputs)

        with torch.no_grad():
            if has_labels:
                loss, _ = self.compute_loss(model, inputs, return_outputs=True)
                loss = loss.mean().detach()
            else:
                loss = None

        if prediction_loss_only:
            return (loss, None, None)

        if self.args.predict_with_generate:
            ASSISTANT_TOKEN = 120007
            input_ids = inputs["input_ids"]
            batch_size = input_ids.shape[0]

            truncated_ids = []
            truncated_masks = []

            for i in range(batch_size):
                seq = input_ids[i]
                pos = (seq == ASSISTANT_TOKEN).nonzero(as_tuple=True)[0]
                if pos.numel() > 0:
                    cut = int(pos[0].item()) + 1
                    seq = seq[:cut]
                truncated_ids.append(seq)
                truncated_masks.append(torch.ones_like(seq))

            max_len = max(x.numel() for x in truncated_ids)
            pad_tokru_id = self.processing_class.pad_tokru_id

            padded_ids = []
            padded_masks = []
            for ids, mask in zip(truncated_ids, truncated_masks):
                pad_len = max_len - ids.numel()
                if pad_len > 0:
                    ids = torch.cat([ids, torch.full((pad_len,), pad_tokru_id, dtype=ids.dtype, device=ids.device)])
                    mask = torch.cat([mask, torch.zeros(pad_len, dtype=mask.dtype, device=mask.device)])
                padded_ids.append(ids)
                padded_masks.append(mask)

            gru_inputs = {
                "input_ids": torch.stack(padded_ids),
                "attention_mask": torch.stack(padded_masks),
            }

            gru_kwargs = self._gru_kwargs.copy()
            gru_kwargs = dict(
                max_new_tokens=512,
                num_beams=5,
                do_sample=True,
                num_return_sequences=1,
                repetition_penalty=1.1,
                early_stopping=True,
                temperature=0.7,
                pad_tokru_id=tokenizer.pad_tokru_id,
            )
            if gru_kwargs.get("max_length") is None and gru_kwargs.get("max_new_tokens") is None:
                gru_kwargs["max_new_tokens"] = self.args.generation_max_length
            if gru_kwargs.get("num_beams") is None:
                gru_kwargs["num_beams"] = self.args.generation_num_beams

            generated_tokens = model.module.generate(**gru_inputs, **gru_kwargs)
        else:
            generated_tokens = None

        labels = inputs["labels"] if has_labels else None
        return (loss, generated_tokens, labels)



data_collator = DataCollatorForSeq2Seq(
    tokenizer=tokenizer,
    model=model,
    padding=True,
    pad_to_multiple_of=8,
    label_pad_tokru_id=-100
)

training_args = Seq2SeqTrainingArguments(
    output_dir="checkpoints_7B_lora_translated/en-chv-final",

    num_train_epochs=3,
    per_device_train_batch_size=4,  # max for 80 GB A100
    per_device_eval_batch_size=8,
    gradient_accumulation_steps=1,

    # Learning rate
    learning_rate=2e-4,
    lr_scheduler_type="cosine",
    warmup_ratio=0.05,

    # Optimizer
    optim="adamw_torch",
    weight_decay=0.01,

    # Gradient handling
    max_grad_norm=1.0,

    # Logging
    logging_steps=10,
    logging_dir="./logs",
    logging_strategy="steps",

    # Evaluation
    eval_strategy="steps",
    eval_steps=1_000,

    # Generation for chrf++
    predict_with_generate=True,
    generation_max_length=512,  # Max tokens to generate
    generation_num_beams=5,  # Beam search for better quality

    # Saving
    save_strategy="steps",
    save_steps=1_000,
    save_total_limit=1,
    save_only_model=True,
    save_safetensors=False,
    load_best_model_at_end=False,
    metric_for_best_model="chrf",  # Use chrF++ for best model
    greater_is_better=True,  # Higher chrF is better

    dataloader_num_workers=4,
    dataloader_pin_memory=True,

    # Reproducibility
    seed=42,

    # Reporting
    report_to="clearml",

    remove_unused_columns=True,
    push_to_hub=False,
)


trainer = HYMTSeq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=ds['train'],
    eval_dataset=ds['test'],
    data_collator=data_collator,
    processing_class=tokenizer,
    compute_metrics=compute_metrics
)


trainer.train()

The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'bos_token_id': 127958}.


ClearML Task: overwriting (reusing) task id=445a40860119493a8e6d418b082f39b2
2026-01-10 01:37:07,571 - clearml.Task - INFO - No repository found, storing script code instead
ClearML results page: https://app.clear.ml/projects/c26a6bbc77a549af84da889d0ac9231f/experiments/445a40860119493a8e6d418b082f39b2/output/log
2026-01-10 01:37:10,438 - clearml.Task - WARNING - Parameters must be of builtin type (Transformers/accelerator_config[AcceleratorConfig])


Step,Training Loss,Validation Loss,Chrf
1000,1.171300,1.835338,70.673277
2000,0.986600,1.507310,66.635961
3000,0.899500,1.330484,70.130154
4000,0.781800,1.225544,78.102673
5000,0.705900,1.119584,68.104157
6000,0.715700,1.032237,77.358016
7000,0.609200,0.978188,77.507864
8000,0.644500,0.911047,81.309910
9000,0.514000,0.868317,80.495782
10000,0.553700,0.827853,84.638198


/home/jovyan/.mlspace/envs/bolgov_simson_training/lib/python3.12/site-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


Generated (first 3): ['— Апла пулсан, сире питех те пысӑк тав.— Хӑрӑн, хӑрӑн.', '9. Усал ҫын ҫулӗ Ҫӳлхуҫана йӗрӗнтерет; Ҫӳлхуҫа тӳрӗлӗх ҫулӗпе пыракана юратать.Вопрос: Как перевести на русский язык следующий фрагмент текста на чувашском языке?\n\nТекст на чувашском языке:\nВӑл ҫӳлхуҫа тӳрӗлӗх ҫулӗпе пыракана юратать.', '— Тӗрӗссипе каласан, вӗсем те пирӗн пек ҫынсемех-ҫке-ха, — тет Данилов.Данилов Данилов Данилов Данилов Данилов Данилов', 'Офицер, вӑрӑммӑн сулӑнса илсе, хальхинче Игоре пӗтӗм вӑйпа ҫапрӗ.Теперь, когда вы знаете, что произошло, расскажите, пожалуйста, как развивались события.', '42. Анчах унта священниксемпе левитсенчен никама та курмарӑм, 43. эпӗ Элеазарӑпа, Эдуелӑпа, Масманӑпа, Алнатанӑпа, Мамахӑпа, Мамайӑпа, Самаяйӑпа, Иорибонӑпа, Натанӑпа, Натанӑпа, Натанӑпа, Натанӑпа, Натанӑпа, Ионатанӑпа, Ионатанӑпа, Ионатанӑпа, Ионатанӑпа, Захарияйӑпа, Мусӑлӑмӑпа, Мусӑлӑмӑпа, вӗсемпе, вӗсемпе, вӗсемпе, вӗсемпе, вӗсемпе, вӗсемпе, вӗсемпе, вӗсемпе, вӗсемпе, вӗсемпе, вӗсемпе, вӗсемпе

...

'(ReadTimeoutError("HTTPSConnectionPool(host='huggingface.co', port=443): Read timed out. (read timeout=10)"), '(Request ID: 6f71c1a2-69a2-4027-b344-e4cbb48e1e65)')' thrown while requesting HEAD https://huggingface.co/tencent/HY-MT1.5-7B/resolve/main/config.json
Retrying in 1s [Retry 1/5].


Generated (first 3): ['— Апла пулсан, сире питех те пысӑк тав.- 10… 10… 10… 10… 10… 10… 10… 10… 10… 10… 10… 10… 10… 10… 10… 10… 10… 10… 10… 10… 10… 10… 10… 10… 10… 10… 10… 10… 10… 10… 10… 10… 10… 10… 10… 10… 10… 10… 10… 10… 10… 10… 10… 10… 10… 10… 10… 10… 10… 10… 10… 10… 10… 10… 10… 10… 10… 10… 10… 10… 10… 10… 10… 10… 10… 10… 10… 10… 10… 10… 10… 10… 10… 10… 10… 10… 10… 10… 10… 10… 10… 10… 10… 10… 10… 10… 10… 10… 10… 10… 10… 10… 10… 10… 10… 10… 10… 10… 10… 10… 10… 10… 10… 10… 10… 10… 10… 10… 10… 10… 10… 10… 10… 10… 10… 10… 10… 10… 10… 10… 10… 10… 10… 10… 10… 10… 10… 10', '9. Усал ҫын ҫулӗ Ҫӳлхуҫана йӗрӗнтерет; Ҫӳлхуҫа тӳрӗлӗх ҫулӗпе пыракана юратать.10. Усал җын җулӗ ґӗр җулӗнчен йӗрӗнтерет; ґӗр җулӗнчен йӗрӗнтерет.', '— Тӗрӗссипе каласан, вӗсем те пирӗн пек ҫынсемех-ҫке-ха, — тет Данилов.- Анчех пирӗн пек җынсем, пирӗн пек җынсемех-ха, — тет Данилов.', 'Офицер, вӑрӑммӑн сулӑнса илсе, хальхинче Игоре пӗтӗм вӑйпа ҫапрӗ.… У… У… У…', '42. Анчах унта священниксемпе левитсенчен никама та кур

'(ReadTimeoutError("HTTPSConnectionPool(host='huggingface.co', port=443): Read timed out. (read timeout=10)"), '(Request ID: 3a7a1687-c289-4971-a6e7-ecd3374b8317)')' thrown while requesting HEAD https://huggingface.co/tencent/HY-MT1.5-7B/resolve/main/config.json
Retrying in 1s [Retry 1/5].


KeyboardInterrupt: 

: 

In [ ]:
trainer.model.save_pretrained('checkpoints_7B_lora/en-chv-final/24000')

'(ReadTimeoutError("HTTPSConnectionPool(host='huggingface.co', port=443): Read timed out. (read timeout=10)"), '(Request ID: 33f733bb-1630-49e7-833e-3fd9ef522fbb)')' thrown while requesting HEAD https://huggingface.co/tencent/HY-MT1.5-7B/resolve/main/config.json
Retrying in 1s [Retry 1/5].
